<a href="https://colab.research.google.com/github/RealPapaZeus/News-Scraper/blob/main/news_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

In [3]:
import os

os.environ["AGENTQL_API_KEY"]="efeqQVTHCP2KpSsoi49ay0rp-Okv92Q-ZzZ3uxT1aZITZLDK7L7pTg"

In [4]:
!pip install agentql playwright nest_asyncio pandas openpyxl
!pip install playwright
!playwright install
!playwright install-deps

Installing dependencies...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building depe

# Load Data

In [11]:
import pandas as pd
import agentql
from playwright.async_api import async_playwright
import asyncio
import json

# Function to load the Excel file
def load_companies_data(excel_file_path):
    df = pd.read_excel(excel_file_path)
    return df

# Scrape News Room

In [12]:
# Function to scrape news articles for each company
async def scrape_news_articles_for_company(company, url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch()
        page = await agentql.wrap_async(await browser.new_page())

        await page.goto(url)

        SEARCH_QUERY = """
        {
            news_articles
        }
        """

        NEWSROOM_OVERVIEW = """
        {
            news_articles[] {
                title
                body
                date
                link
            }
        }
        """
        # Get the response
        response = await page.query_data(NEWSROOM_OVERVIEW)

        # Print the response to inspect its structure
        print(response)

        # Access the articles from the response, if available
        articles = response.get('news_articles', [])

        await browser.close()

        # Return the company's name and the extracted news articles
        return {
            "company": company,
            "news_articles": articles
        }

# Process Companies Data

In [13]:
# Main function to process the Excel and scrape data
async def process_companies_data(excel_file_path):
    companies_data = load_companies_data(excel_file_path)

    # List to hold the results for all companies
    all_companies_news = []

    # Scrape news articles for each company
    for _, row in companies_data.iterrows():
        company_name = row['company']
        company_url = row['link']
        print(f"Scraping news articles for: {company_name} ({company_url})")

        company_news = await scrape_news_articles_for_company(company_name, company_url)
        all_companies_news.append(company_news)

    # Convert the data to a DataFrame
    df = pd.json_normalize(all_companies_news, "news_articles", ["company"])

    # Convert the DataFrame to JSON for structured output
    structured_json = df.to_json(orient="records", lines=True)

    return df, structured_json

# Execute Code

In [14]:
import asyncio

loop = asyncio.get_running_loop()

excel_file_path = "/content/competitor_newsroom.xlsx"
task = loop.create_task(process_companies_data(excel_file_path))
df, structured_json = await task
# Display DataFrame and JSON result
print(df.head())
print(structured_json)

Scraping news articles for: Accenture (https://newsroom.accenture.com/)
{'news_articles': [{'title': 'Accenture and Schaeffler Pave the Way for Industrial Humanoid Robots with NVIDIA and Microsoft Technologies', 'body': 'HANOVER; Germany; April 1, 2025 – Accenture (NYSE: ACN) is working with Schaeffler AG to reinvent industrial automation with physical AI and robotics.', 'date': 'April 01, 2025', 'link': 'https://newsroom.accenture.com/news/2025/accenture-and-schaeffler-pave-the-way-for-industrial-humanoid-robots-with-nvidia-and-microsoft-technologies'}, {'title': 'Malabar Gold & Diamonds Collaborates with Accenture to Accelerate AI-Driven Business Reinvention', 'body': 'NEW DELHI; April 1, 2025 – Malabar Gold & Diamonds, the sixth largest jewelry retailer globally with over 380 showrooms across 13 countries, is collaborating with Accenture (NYSE: ACN) to accelerate its technology reinvention through an AI-powered, cloud enabled digital core. This initiative aims to elevate the custome

In [15]:
# Save DataFrame to Excel
output_file_path = "/content/competitor_newsroom_output.xlsx"
df.to_excel(output_file_path, index=False, engine='openpyxl')